In [ ]:
from IPython.display import clear_output
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import re

from tqdm import tqdm
import sys
import dropbox
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl
from amftrack.util.dbx import load_dbx
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
)
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
    
)

%matplotlib widget
%load_ext autoreload
%autoreload 2
logging.basicConfig(stream=sys.stdout, level=logging.debug)
mpl.rcParams['figure.dpi'] = 200

print("import great succes!")
%reload_ext autoreload

In [ ]:
analysis_folder = "/gpfs/home6/svstaalduine/Analysis/CocoTransport/"
dropbox_address = "/DATA/CocoTransport/"
excel_address = "/Daan te Rietmole/SpeedCheck.xlx"

analysis_filter = "20230906_Plate310/"

In [ ]:
%%time
# edgedata = glob.glob("/gpfs/home6/svstaalduine/Analysis/CocoTransport/20230906_Plate310/052/Img/*csv")
edgedata = glob.glob(analysis_folder+analysis_filter+"*/*/*.csv")
print(len(edgedata))
print(analysis_folder+analysis_filter+"*/*.csv")

In [ ]:
df1 = pd.read_csv(edgedata[0])
print(df1)

In [ ]:
%%time

list_of_dfs = []

interested_subdirectories = ['20230906_Plate310']
# Walk through the directory and its subdirectories
i=0
for root, dirs, files in os.walk(analysis_folder):
    if any(subdir in root for subdir in interested_subdirectories):
        for file in files:
            if file.endswith('edges_data.csv'):
                
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                jsnpath = file_path[:-14]+'video_data.json'
                 
                with open(jsnpath, 'r') as file:
                    jason = json.load(file)
                 
                df.insert(1,"mode",jason['mode'])
                df.insert(2,"video_int",jason['video_int'])
                 
                list_of_dfs.append(df)
                    

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(list_of_dfs, ignore_index=True)

print(len(combined_df))
print(combined_df.iloc[0])

In [ ]:
print(df.columns.tolist())

In [ ]:
print(combined_df['edge_name'].iloc[2])

In [ ]:
%%time
sel_df = combined_df
# sel_df = combined_df[combined_df['speed_left']<=-4]
# sel_df = sel_df[sel_df['speed_right']>=4]
# sel_df = sel_df[sel_df['coverage_right']>=0.25]
# sel_df = sel_df[sel_df['coverage_left']>=0.25]
# sel_df = sel_df[sel_df['coverage_tot']>=0.7]
# sel_df = sel_df[sel_df['straight_length']<=200]


# sel_df = sel_df[sel_df['std_right']<=0.16]



BF_df = sel_df[sel_df['mode']=='BF']
FL_df = sel_df[sel_df['mode']=='F']

print(len(BF_df))
print(len(FL_df))

# print(BF_df[BF_df['edge_name']=='(24, 5)'])

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(FL_df)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(combined_BF_df['speed_left'],combined_BF_df['speed_right'], alpha=combined_BF_df['coverage_tot'], color='green')
ax.scatter(combined_FL_df['speed_left'],combined_FL_df['speed_right'], alpha=combined_FL_df['coverage_tot'], color='red')
plt.show()

In [ ]:
plt.close('all')
fig, ax = plt.subplots(2,2, figsize=(10, 10))
ax[0][0].set_xlim(-12,0)
ax[0][0].set_ylim(0,12)
ax[1][0].set_xlim(-12,0)
ax[1][0].set_ylim(0,12)
ax[1][1].set_xlim(-12,0)
ax[1][1].set_ylim(0,12)
ax[0][0].scatter(BF_df['speed_left'],BF_df['speed_right'], alpha=BF_df['coverage_left'], color='green')
ax[0][0].scatter(FL_df['speed_left'],FL_df['speed_right'], alpha=FL_df['coverage_left'], color='red')
ax[1][0].scatter(BF_df['speed_left'],BF_df['speed_right'], alpha=BF_df['coverage_right'], color='green')
ax[1][0].scatter(FL_df['speed_left'],FL_df['speed_right'], alpha=FL_df['coverage_right'], color='red')
ax[1][1].scatter(BF_df['speed_left'],BF_df['speed_right'], alpha=BF_df['coverage_tot'], color='green')
ax[1][1].scatter(FL_df['speed_left'],FL_df['speed_right'], alpha=FL_df['coverage_tot'], color='red')
plt.show()

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
ax.scatter(BF_df['speed_left'],BF_df['speed_right'], alpha=BF_df['coverage_tot'], color='green')
ax.scatter(FL_df['speed_left'],FL_df['speed_right'], alpha=FL_df['coverage_tot'], color='red')
plt.show()

1. download the excel by hand analysis.
2. load it into a dataframe.
3. compare it with the automated simon analysis.

In [ ]:
excel_address